In [99]:
import math, copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.metrics import PrecisionRecallDisplay, f1_score, mean_absolute_error, precision_recall_curve, \
    precision_score, recall_score
my_ID = 400132290
np.random.seed(my_ID)
np.set_printoptions(precision=2)# reduced display precision on numpy arrays

In [100]:
data = load_breast_cancer()
x_data = pd.DataFrame(data.data, columns=data.feature_names)
y_data = data.target
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=my_ID)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

print(f"X Shape: {x_train.shape}, X Type:{type(x_train)})")
#print(x_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
#print(y_train)

X Shape: (455, 30), X Type:<class 'numpy.ndarray'>)
y Shape: (455,), y Type:<class 'numpy.ndarray'>)


**to do**
- my version of Batch and SGD
- compute missclassification rate
- F1 ?
- plot precision/recall PR curve and ROC curve
- something about B???

In [101]:
# without matrix multiplication
def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar, numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
         
    """

    g = 1/(1+np.exp(-z))
   
    return g

def compute_cost_logistic(X, y, w, b):
    """
    Computes cost

    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """

    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
             
    cost = cost / m
    return cost

def compute_gradient_logistic(X, y, w, b): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
    Returns
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar)      : The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #scalar
        
    return dj_db, dj_dw 

def gradient_descent(X, y, w_in, b_in, alpha, num_iters): 
    """
    Performs batch gradient descent
    
    Args:
      X (ndarray (m,n)   : Data, m examples with n features
      y (ndarray (m,))   : target values
      w_in (ndarray (n,)): Initial values of model parameters  
      b_in (scalar)      : Initial values of model parameter
      alpha (float)      : Learning rate
      num_iters (scalar) : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,))   : Updated values of parameters
      b (scalar)         : Updated value of parameter 
    """
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
        
    return w, b, J_history         #return final w,b and J history for graphing

def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
    J_history = []
   
    # Loop over each example
    for i in range(m):   
        f_wb_i = sigmoid(np.dot(X[i],w) +b)
        
        if f_wb_i >= 0.5:
            p[i] = 1
        else:
            p[i] = 0
        
    return p

In [102]:
# SGD
def compute_gradient_logistic_SGD(X, y, w, b): 
    """
    Computes the gradient for linear regression 
 
    Args:
      X (ndarray (m,n): Data, m examples with n features
      y (ndarray (m,)): target values
      w (ndarray (n,)): model parameters  
      b (scalar)      : model parameter
    Returns
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar)      : The gradient of the cost w.r.t. the parameter b. 
    """
    n = X.shape[0]
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.


    f_wb_i = sigmoid(np.dot(X,w) + b)          #(n,)(n,)=scalar
    err_i  = f_wb_i  - y                       #scalar
    for j in range(n):
        dj_dw[j] = dj_dw[j] + err_i * X[j]      #scalar
    dj_db = dj_db + err_i
        
    return dj_db, dj_dw  

def gradient_descent_SGD(X, y, w_in, b_in, alpha, num_iters): 
    """
    Performs batch gradient descent
    
    Args:
      X (ndarray (m,n)   : Data, m examples with n features
      y (ndarray (m,))   : target values
      w_in (ndarray (n,)): Initial values of model parameters  
      b_in (scalar)      : Initial values of model parameter
      alpha (float)      : Learning rate
      num_iters (scalar) : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,))   : Updated values of parameters
      b (scalar)         : Updated value of parameter 
    """
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    num_rows = X.shape[0]
    for i in range(num_iters):
        # generate random index and index the array to retrieve a roll
        random_index = np.random.choice(num_rows)
        
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic_SGD(X[random_index], y[random_index], w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
        
    return w, b, J_history         #return final w,b and J history for graphing

# X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
# y_train = np.array([460, 232, 178])
# b_init = 785.1811367994083
# w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
#Compute and display gradient 

# tmp_dj_db, tmp_dj_dw = compute_gradient_logistic_SGD(X_train[i], y_train[i], w_init, b_init)
# print(f'dj_db at initial w,b: {tmp_dj_db}')
# print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')


In [103]:
# w_tmp  = np.zeros_like(x_train[0])
# b_tmp  = 0.
# alph = 0.1
# iters = 1000
# w_out, b_out, _ = gradient_descent_SGD(x_train, y_train, w_tmp, b_tmp, alph, iters)

# p_train = predict(x_train, w_out, b_out)
# p_test = predict(x_test, w_out, b_out)
# print(f'Output of predict: shape {p_test.shape}')
# print('Train Accuracy: %f'%(np.mean(p_train == y_train) * 100))
# print('Test Accuracy: %f'%(np.mean(p_test == y_test) * 100))
# print(f"model parameters (SGD): w: {w_out}, b:{b_out}")

In [104]:
# w_tmp  = np.zeros_like(x_train[0])
# b_tmp  = 0.
# alph = 0.1
# iters = 1000

# w_out, b_out, _ = gradient_descent(x_train, y_train, w_tmp, b_tmp, alph, iters)

# p_train = predict(x_train, w_out, b_out)
# p_test = predict(x_test, w_out, b_out)
# print(f'Output of predict: shape {p_test.shape}')
# print('Train Accuracy: %f'%(np.mean(p_train == y_train) * 100))
# print('Test Accuracy: %f'%(np.mean(p_test == y_test) * 100))
# print(f"model parameters (batch): w: {w_out}, b:{b_out}")

In [105]:
# matrix implementation
def train_logistic_regression(x_train, y_train, a):
    """
    Perform batch gradient descent to train the logistic regression model

    :param x_train: the training input
    :param y_train: the true values
    :param tolerance: the threshold at which to stop training
    :param lr: the learning rate
    :return: the weights and bias for the model
    """
    # initialize weights and bias to zero
    weights = np.zeros(x_train.shape[1])
    i = 0
    while True:
        # calculate predictions
        preds = sigmoid(x_train @ weights)

        # update the params
        grad = (1 / x_train.shape[0]) * x_train.T @ (preds - y_train)

        weights -= grad * a

        # calculate the loss
        loss = y_train * np.logaddexp(0, -(x_train @ weights)) + (1 - y_train) * np.logaddexp(0, -(x_train @ weights))

        i += 1

        if i > 1000:
            break

    return weights

def train_logistic_regression_SGD(x_train, y_train, a):
    """
    Perform batch gradient descent to train the logistic regression model

    :param x_train: the training input
    :param y_train: the true values
    :param tolerance: the threshold at which to stop training
    :param lr: the learning rate
    :return: the weights and bias for the model
    """
    # initialize weights and bias to zero
    weights = np.zeros(x_train.shape[1])
    i = 0
    num_rows = x_train.shape[0]
    while True:
        # randomly p
        random_index = np.random.choice(num_rows)
        x_train_SGD = x_train[random_index]
        # calculate predictions
        preds = sigmoid(x_train_SGD @ weights)

        # update the params
        grad = x_train_SGD.T * (preds - y_train[random_index])

        weights -= grad * a

        # calculate the loss
        #loss = y_train * np.logaddexp(0, -(x_train @ weights)) + (1 - y_train) * np.logaddexp(0, -(x_train @ weights))

        i += 1

        if i > 1000:
            break

    return weights

def predict_log_reg(x_test, weights, threshold=0.5):
    preds = sigmoid(x_test @ weights)

    # split classification at threshold
    pred_class = [1 if i > threshold else 0 for i in preds]

    return np.array(pred_class)


# initialize data
x_train_mat = np.c_[np.ones(x_train.shape[0]), x_train]
x_test_mat = np.c_[np.ones(x_test.shape[0]), x_test]
w_batch = train_logistic_regression(x_train_mat, y_train, 0.1)
print(f"log reg {w_batch}","\n")

p_train = predict_log_reg(x_train_mat, w_batch)
p_test = predict_log_reg(x_test_mat, w_batch)
print('Train Accuracy: %f'%(np.mean(p_train == y_train) * 100))
print('Test Accuracy: %f'%(np.mean(p_test == y_test) * 100))

w_batch = train_logistic_regression_SGD(x_train_mat, y_train, 0.1)
print(f"log reg SGD {w_batch}","\n")

p_train = predict_log_reg(x_train_mat, w_batch)
p_test = predict_log_reg(x_test_mat, w_batch)
print('Train Accuracy: %f'%(np.mean(p_train == y_train) * 100))
print('Test Accuracy: %f'%(np.mean(p_test == y_test) * 100))

log reg [ 0.23 -0.6  -0.68 -0.59 -0.6  -0.23  0.17 -0.66 -0.75  0.06  0.47 -0.82
  0.01 -0.62 -0.65 -0.19  0.5   0.06 -0.14  0.41  0.47 -0.84 -1.05 -0.77
 -0.78 -0.79 -0.13 -0.79 -0.76 -0.69 -0.14] 

Train Accuracy: 98.901099
Test Accuracy: 95.614035
log reg SGD [ 0.51 -0.58 -0.79 -0.57 -0.61 -0.44  0.09 -0.76 -0.98  0.2   0.33 -1.11
 -0.07 -0.85 -0.85 -0.13  0.68  0.04 -0.17  0.41  0.43 -0.84 -1.15 -0.77
 -0.83 -0.82  0.07 -0.68 -0.94 -0.52 -0.07] 

Train Accuracy: 98.021978
Test Accuracy: 95.614035


In [106]:
# sklearn
# batch
log_reg_batch =LogisticRegression(random_state=my_ID)
log_reg_batch.fit(x_train, y_train)
print(f"number of iterations completed: {log_reg_batch.n_iter_}")
log_reg_batch_b = log_reg_batch.intercept_
log_reg_batch_w = log_reg_batch.coef_
print(f"model parameters (batch): w: {log_reg_batch_w}, b:{log_reg_batch_b}")

predictions_batch = log_reg_batch.predict(x_test)
#print(f"Prediction on training set (batch):\n{predictions_batch[:10]}" )
#print(f"Target values (batch)\n{y_test[:10]}")

best_misclass_rate = mean_absolute_error(y_test, predictions_batch)
f1_s = f1_score(y_test, predictions_batch)
print(f"The test error for the sklearn implementation is: {best_misclass_rate}, the f1-score is: {f1_s}")


# SGD
log_reg_SGD = SGDClassifier(random_state=my_ID)
log_reg_SGD.fit(x_train, y_train)
print(f"number of iterations completed: {log_reg_SGD.n_iter_}")
log_reg_SGD_b = log_reg_SGD.intercept_
log_reg_SGD_w = log_reg_SGD.coef_
print(f"model parameters (SGD): w: {log_reg_SGD_w}, b:{log_reg_SGD_b}")

predictions_SGD = log_reg_SGD.predict(x_test)
#print(f"Prediction on training set (SGD):\n{predictions_SGD[:10]}" )
# predictions_round = [0 if num < 0.5 else 1 for num in predictions]
# print(f"Prediction on training set rounded (SGD):\n{predictions_round[:10]}" )
#print(f"Target values (SGD)\n{y_test[:10]}")

best_misclass_rate = mean_absolute_error(y_test, predictions_SGD)
f1_s = f1_score(y_test, predictions_SGD)
print(f"The test error for the sklearn implementation is: {best_misclass_rate}, the f1-score is: {f1_s}")

number of iterations completed: [32]
model parameters (batch): w: [[-0.48 -0.46 -0.47 -0.5  -0.09  0.56 -0.9  -1.    0.17  0.31 -1.11  0.12
  -0.72 -0.87 -0.33  0.59  0.2  -0.24  0.49  0.48 -0.89 -1.28 -0.76 -0.87
  -0.75  0.08 -1.08 -0.71 -0.94 -0.21]], b:[-0.03]
The test error for the sklearn implementation is: 0.03508771929824561, the f1-score is: 0.9746835443037976
number of iterations completed: 36
model parameters (SGD): w: [[-1.85  0.52 -1.86 -1.75 -1.46  9.83 -6.44 -6.09  2.91 -2.29 -9.15  2.19
  -2.9  -8.67 -3.8  -1.04  9.36 -4.68  3.9   4.85 -4.52 -9.59 -2.7  -5.87
  -1.1   0.76 -8.23 -1.82 -6.26  0.27]], b:[-3.19]
The test error for the sklearn implementation is: 0.03508771929824561, the f1-score is: 0.975
